In [2]:
import numpy
import pandas as pd
import csv
#import matplotlib.pyplot as plt
#import os

In [3]:
#experiment_date='2020-02-17T164323'
#folder_results='results'
#file_prefix='results-'
file_name='traces.csv'

In [4]:
#file=folder_results+'/'+file_prefix+experiment_date+'/'+file_name
cassandra_traces = pd.read_csv(file_name)
tracedata = pd.DataFrame(cassandra_traces)
#tracedata.head(5)

1. Identify root traces
1. For each root trace, find children
1. Merge transparency tags as "Sets"
1. Map tags to vocabulary

In [16]:
#tracedata.describe()
printable = tracedata.sort_values(by = ['trace_id']).reset_index(drop=True)
#print(printable.dtypes)
#conv = printable.astype({'span_id': 'object'})
printable['span_id'] = printable['span_id'].apply(str)
print(printable.head(9).to_latex(buf = 'tex-table.txt', columns = ['trace_id', 'span_id', 'duration', 'operation_name', 'tags'], formatters = {'trace_id': lambda x: x[-14:], 'span_id': lambda x: x[-5:]}))
relevant = tracedata.filter(items=['trace_id','span_id','operation_name', 'refs', 'tags'])
relevant['is_root'] = relevant['refs'].isna()
relevant = relevant.sort_values(by = ['trace_id', 'is_root']).reset_index(drop=True)
#relevant.head()
#print(relevant.head(5).to_latex())

None


### Exporting from cassandra with JSON column
As it turns out, cassandra has its own internal model for JSON-formatted columns, which on export to CSV, are not properly formatted JSON (missing double quotes for keys). For SELECT queries in CQL, there is a JSON modifier, that fixes the output for columns, but there is none for CQL's COPY.

- [x] TODO: find a way to parse the column of the CSV or export manually by running SELECT and writing results..?
    - Solution: use regexp parsing (lol)
- [x] TODO: re-generate output and CSV, so purposes are separated by something else than comma, so we can regexp-parse the malformed JSON output easier.
    - used semicolon

In [5]:
import re
for i, row in relevant.iterrows():
    #preprocessing...
    relevant.at[i, 'refs'] = '{"refs": '+re.sub("(\w+):\s'?([\w*\-.;:\w*]*)'?", r'"\1": "\2"', str(row.refs))+'}'
    relevant.at[i, 'tags'] = '{"tags": '+re.sub("(\w+):\s'?([\w*\-.;:\w*]*)'?", r'"\1": "\2"', str(row.tags))+'}'
#relevant['tags'].head(5)[1]

In [6]:
import json
## traverse dict a and merge dict b
def load_dict(val):
    #TODO: later remove artificially added key in preprocessing, so json.loads gets us a list of objects directly
    dict_raw = json.loads(val)
    #flatten dict
    listOfObjects = dict_raw['tags']
    new = {}
    for obj in listOfObjects:
        if len(obj['value_string'].split(';')) > 1:
            new[obj['key']] = []
            for item in obj['value_string'].split(';'):
                new[obj['key']].append(item)
        else:
            new[obj['key']] = obj['value_string']
    return new

#based on: https://stackoverflow.com/questions/7204805/how-to-merge-dictionaries-of-dictionaries/7205107#7205107
def deep_merge(a, b_raw, path=None):
    "merges b into a"
    b = load_dict(b_raw)
    if path is None: path = []
    for key in b:
        if key in a:
            if type(a[key]) is not list:
                a[key] = [a[key]]
            if type(b[key]) is list:
                for val in b[key]:
                    a[key].append(val)
            else:
                a[key].append(b[key])
        else:
            a[key] = b[key]
    return a

In [7]:
#for i, row in relevant.head(10).iterrows():
 #   objects = load_dict(row.tags)
 #   print(objects)

In [17]:
trace_id_old = ''
tagset = {}
tagsets = {}
for i, row in relevant.iterrows():
    if row.trace_id == trace_id_old:
        tagset = deep_merge(tagset, row.tags)
    else:
        #remove duplicates from list
        for key, val in tagset.items():
            if type(val) is list:
                tagset[key] = list(set(val))
        tagsets[trace_id_old] = tagset
        tagset = load_dict(row.tags)
        trace_id_old = row.trace_id

In [18]:
tagsets

{'': {},
 '0x0000000000000000002c91df57b25d13': {'categories-processing': ['',
   '03',
   '00',
   '04',
   '01'],
  'categories-storage': ['00', '04', '03', '01'],
  'storage-ttl': ['00', '01'],
  'recipients': ['', '001', '000'],
  'automation': [''],
  'purposes': ['00', '02', '01'],
  'legal-basis': ['00', '02', '01'],
  'legitimate-interest': ['', '02', '01'],
  'internal.span.format': ['proto'],
  'sources': ['', '000'],
  'sampler.type': 'probabilistic',
  'sampler.param': ''},
 '0x00000000000000000042db74fbb127d9': {'internal.span.format': ['proto'],
  'purposes': ['00', '02', '01'],
  'legal-basis': ['00', '02', '01'],
  'categories-processing': ['', '03', '00', '04', '01'],
  'categories-storage': ['00', '04', '03', '01'],
  'storage-ttl': ['00', '01'],
  'recipients': ['', '001', '000'],
  'sources': ['', '000'],
  'automation': [''],
  'legitimate-interest': ['', '02', '01'],
  'sampler.type': 'probabilistic',
  'sampler.param': ''},
 '0x0000000000000000004831cb29bc787a': 

## TODO
* There might be something wrong with the combined data, e.g. traces which only contain recipients tags should not exist as this would imply that the facebook-service is called on its own
  * This fixed itself after a restart??????

In [48]:
with open('vocab.json', 'r') as vocab_file:
    vocab = json.load(vocab_file)
vocab

{'categories': {'00': {'name': 'Identity Information',
   'description': 'First and Last Name(s), date of birth, unique account identifier, gender, social media identifiers'},
  '01': {'name': 'Location Information', 'description': ''},
  '02': {'name': 'Size Information', 'description': ''},
  '03': {'name': 'Activity Information', 'description': ''},
  '04': {'name': 'Social Media Information', 'description': ''}},
 'purposes': {'00': {'name': 'Prodividing of Service', 'description': ''},
  '01': {'name': 'Business'},
  '02': {'name': 'Legal Compliance'}},
 'ttl': {'00': {'name': 'Account Lifetime', 'description': ''},
  '01': {'name': 'Unlimited', 'description': ''},
  '02': {'name': 'Processing Only', 'description': ''}},
 'third-parties': {'000': {'name': 'facebook Europe - Friends API, incoming',
   'address': {'office': '',
    'country': '',
    'zip': '',
    'street': '',
    'number': '',
    'additional': ''}},
  '001': {'name': 'facebook Europe - Friends API, outgoing',
  

## From Traces, Lookup Values in the Vocab
* iterate over traces
    * iterate over transparency keys
        * for each key, lookup mapping in translation
        * for each key, lookup key in vocab
        * in category of vocab, lookup "id" for each item in list of values, add "name" field to policy.
        * if we get an error (e.g., a missing key), this hints at an issue in our policy description
    


In [49]:
lookup_translation = {
    "recipients": "third-parties",
    "sources": "third-parties",
    "legal-basis": "legal-bases",
    "categories-processing": "categories",
    "categories-storage": "categories",
    "categories-storage": "categories",
    "storage-ttl": "ttl",
    "legitimate-interest": "legitimate-interests"
}

In [61]:
request_policies = {}
for traceid in tagsets.keys():
    tmp_policy = {}
    for cat_key, cat_value in tagsets[traceid].items():
        category = cat_key
        if cat_key in lookup_translation:
            category = lookup_translation[cat_key]
        if type(cat_value) is list:
            tmp_policy[cat_key] = []
            if category in vocab.keys():
                for id_val in cat_value:
                    if id_val != "":
                        tmp_policy[cat_key].append(vocab[category][id_val]['name'])
            else:
                if category in vocab.keys():
                    tmp_policy[cat_key] = vocab[category][cat_value]['name']
    #print(tmp_policy)
    request_policies[traceid] = tmp_policy

In [62]:
request_policies

{'': {},
 '0x0000000000000000002c91df57b25d13': {'categories-processing': ['Activity Information',
   'Identity Information',
   'Social Media Information',
   'Location Information'],
  'categories-storage': ['Identity Information',
   'Social Media Information',
   'Activity Information',
   'Location Information'],
  'storage-ttl': ['Account Lifetime', 'Unlimited'],
  'recipients': ['facebook Europe - Friends API, outgoing',
   'facebook Europe - Friends API, incoming'],
  'automation': [],
  'purposes': ['Prodividing of Service', 'Legal Compliance', 'Business'],
  'legal-basis': ['Consent', 'Legitimate Interest', 'Contractual'],
  'legitimate-interest': ['Business Improvement', 'Compliance'],
  'internal.span.format': [],
  'sources': ['facebook Europe - Friends API, incoming']},
 '0x00000000000000000042db74fbb127d9': {'internal.span.format': [],
  'purposes': ['Prodividing of Service', 'Legal Compliance', 'Business'],
  'legal-basis': ['Consent', 'Legitimate Interest', 'Contractua